In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import neighbors
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")

In [20]:
file_path = r"D:\STUDIES\ML\Projects\Projects\Data_Science_Playground\baseline_data.pqt"
df = pd.read_parquet(file_path, engine='fastparquet')
df

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second
0,27226,3,1,13,120,2017-11-06 15:13:23,None,0,6,15,13,23
1,110007,35,1,13,10,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,15,41,7
2,1047,6,1,13,157,2017-11-06 15:42:32,None,0,6,15,42,32
3,76270,3,1,13,120,2017-11-06 15:56:17,None,0,6,15,56,17
4,57862,3,1,13,120,2017-11-06 15:57:01,None,0,6,15,57,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2300556,9791,2,1,19,166,2017-11-09 15:59:59,None,0,9,15,59,59
2300557,6240,14,1,13,146,2017-11-09 15:59:59,None,0,9,15,59,59
2300558,15098,12,2,17,50,2017-11-09 16:00:00,None,0,9,16,0,0
2300559,10538,12,1,15,41,2017-11-09 16:00:00,None,0,9,16,0,0


In [21]:
df['time_to_conversion'] = (pd.to_datetime(df['attributed_time']) - pd.to_datetime(df['click_time'])).dt.total_seconds()

In [10]:
df

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,time_to_conversion
0,27226,3,1,13,120,2017-11-06 15:13:23,None,0,6,15,13,23,NaN
1,110007,35,1,13,10,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,15,41,7,59772.0
2,1047,6,1,13,157,2017-11-06 15:42:32,None,0,6,15,42,32,NaN
3,76270,3,1,13,120,2017-11-06 15:56:17,None,0,6,15,56,17,NaN
4,57862,3,1,13,120,2017-11-06 15:57:01,None,0,6,15,57,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300556,9791,2,1,19,166,2017-11-09 15:59:59,None,0,9,15,59,59,NaN
2300557,6240,14,1,13,146,2017-11-09 15:59:59,None,0,9,15,59,59,NaN
2300558,15098,12,2,17,50,2017-11-09 16:00:00,None,0,9,16,0,0,NaN
2300559,10538,12,1,15,41,2017-11-09 16:00:00,None,0,9,16,0,0,NaN


In [14]:
df2 = df[df["is_attributed"]==1]
df2

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,time_to_conversion
1,110007,35,1,13,10,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,15,41,7,59772.0
15,8981,9,1,22,60,2017-11-06 16:00:02,2017-11-07 10:05:22,1,6,16,0,2,65120.0
19,19577,35,1,13,10,2017-11-06 16:00:02,2017-11-06 23:40:50,1,6,16,0,2,27648.0
23,80732,35,1,45,86,2017-11-06 16:00:03,2017-11-07 00:55:29,1,6,16,0,3,32126.0
36,105289,35,1,13,86,2017-11-06 16:00:04,2017-11-06 23:04:54,1,6,16,0,4,25490.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300297,212381,10,1,27,20,2017-11-09 15:59:21,2017-11-09 15:59:53,1,9,15,59,21,32.0
2300298,228807,5,1,46,20,2017-11-09 15:59:21,2017-11-09 15:59:26,1,9,15,59,21,5.0
2300328,9009,45,1,32,137,2017-11-09 15:59:26,2017-11-09 15:59:42,1,9,15,59,26,16.0
2300334,207460,10,1,11,20,2017-11-09 15:59:26,2017-11-09 15:59:58,1,9,15,59,26,32.0


In [22]:
df['device_os_combination'] = df['device'].astype(str) + "_" + df['os'].astype(str)
device_os_counts = df['device_os_combination'].value_counts()
df['device_os_count'] = df['device_os_combination'].map(device_os_counts)
df['ip_click_count'] = df.groupby('ip')['click_time'].transform('count')
df['channel_click_count'] = df.groupby('channel')['click_time'].transform('count')
df

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,time_to_conversion,device_os_combination,device_os_count,ip_click_count,channel_click_count
0,27226,3,1,13,120,2017-11-06 15:13:23,None,0,6,15,13,23,NaN,1_13,443212,78,35101
1,110007,35,1,13,10,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,15,41,7,59772.0,1_13,443212,4,45033
2,1047,6,1,13,157,2017-11-06 15:42:32,None,0,6,15,42,32,NaN,1_13,443212,150,35115
3,76270,3,1,13,120,2017-11-06 15:56:17,None,0,6,15,56,17,NaN,1_13,443212,35,35101
4,57862,3,1,13,120,2017-11-06 15:57:01,None,0,6,15,57,1,NaN,1_13,443212,40,35101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300556,9791,2,1,19,166,2017-11-09 15:59:59,None,0,9,15,59,59,NaN,1_19,509743,240,72923
2300557,6240,14,1,13,146,2017-11-09 15:59:59,None,0,9,15,59,59,NaN,1_13,443212,312,835
2300558,15098,12,2,17,50,2017-11-09 16:00:00,None,0,9,16,0,0,NaN,2_17,4917,499,53041
2300559,10538,12,1,15,41,2017-11-09 16:00:00,None,0,9,16,0,0,NaN,1_15,46898,60,41066


In [23]:
df = df.sort_values(['ip', 'click_time'])
df['prev_click_time'] = df.groupby('ip')['click_time'].shift(1)
df['time_since_last_click'] = (pd.to_datetime(df['click_time']) - pd.to_datetime(df['prev_click_time'])).dt.total_seconds()

In [24]:
df

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,time_to_conversion,device_os_combination,device_os_count,ip_click_count,channel_click_count,prev_click_time,time_since_last_click
1705658,0,10,1,46,20,2017-11-09 01:43:08,2017-11-09 01:43:42,1,9,1,43,8,34.0,1_46,19572,9,60114,NaT,NaN
1706762,0,10,1,42,20,2017-11-09 01:44:53,2017-11-09 01:45:27,1,9,1,44,53,34.0,1_42,8749,9,60114,2017-11-09 01:43:08,105.0
2169455,0,10,1,4,20,2017-11-09 12:45:57,2017-11-09 12:46:29,1,9,12,45,57,32.0,1_4,11043,9,60114,2017-11-09 01:44:53,39664.0
2170017,0,10,1,30,20,2017-11-09 12:46:46,2017-11-09 12:46:50,1,9,12,46,46,4.0,1_30,16714,9,60114,2017-11-09 12:45:57,49.0
2182338,0,5,1,30,20,2017-11-09 13:03:42,2017-11-09 13:04:25,1,9,13,3,42,43.0,1_30,16714,9,60114,2017-11-09 12:46:46,1016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938032,260234,19,5,29,59,2017-11-09 06:58:55,2017-11-09 06:59:37,1,9,6,58,55,42.0,5_29,3029,1,139698,NaT,NaN
1972339,260235,34,1,19,73,2017-11-09 07:48:05,2017-11-09 07:51:09,1,9,7,48,5,184.0,1_19,509743,2,8848,NaT,NaN
2209994,260235,3,1,22,13,2017-11-09 13:41:51,None,0,9,13,41,51,NaN,1_22,85200,2,2052,2017-11-09 07:48:05,21226.0
2197432,260236,19,0,21,59,2017-11-09 13:24:39,2017-11-09 13:25:29,1,9,13,24,39,50.0,0_21,15215,1,139698,NaT,NaN


In [27]:
df['is_weekend'] = df['day'].isin([6, 7]).astype(int)
df['is_night'] = ((df['hour'] >= 0) & (df['hour'] < 6)).astype(int)
df

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,...,second,time_to_conversion,device_os_combination,device_os_count,ip_click_count,channel_click_count,prev_click_time,time_since_last_click,is_weekend,is_night
1705658,0,10,1,46,20,2017-11-09 01:43:08,2017-11-09 01:43:42,1,9,1,...,8,34.0,1_46,19572,9,60114,NaT,NaN,0,1
1706762,0,10,1,42,20,2017-11-09 01:44:53,2017-11-09 01:45:27,1,9,1,...,53,34.0,1_42,8749,9,60114,2017-11-09 01:43:08,105.0,0,1
2169455,0,10,1,4,20,2017-11-09 12:45:57,2017-11-09 12:46:29,1,9,12,...,57,32.0,1_4,11043,9,60114,2017-11-09 01:44:53,39664.0,0,0
2170017,0,10,1,30,20,2017-11-09 12:46:46,2017-11-09 12:46:50,1,9,12,...,46,4.0,1_30,16714,9,60114,2017-11-09 12:45:57,49.0,0,0
2182338,0,5,1,30,20,2017-11-09 13:03:42,2017-11-09 13:04:25,1,9,13,...,42,43.0,1_30,16714,9,60114,2017-11-09 12:46:46,1016.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938032,260234,19,5,29,59,2017-11-09 06:58:55,2017-11-09 06:59:37,1,9,6,...,55,42.0,5_29,3029,1,139698,NaT,NaN,0,0
1972339,260235,34,1,19,73,2017-11-09 07:48:05,2017-11-09 07:51:09,1,9,7,...,5,184.0,1_19,509743,2,8848,NaT,NaN,0,0
2209994,260235,3,1,22,13,2017-11-09 13:41:51,None,0,9,13,...,51,NaN,1_22,85200,2,2052,2017-11-09 07:48:05,21226.0,0,0
2197432,260236,19,0,21,59,2017-11-09 13:24:39,2017-11-09 13:25:29,1,9,13,...,39,50.0,0_21,15215,1,139698,NaT,NaN,0,0


In [34]:
list1 = df.columns.to_list
print(list1)

<bound method IndexOpsMixin.tolist of Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed', 'day', 'hour', 'minute', 'second',
       'time_to_conversion', 'device_os_combination', 'device_os_count',
       'ip_click_count', 'channel_click_count', 'prev_click_time',
       'time_since_last_click', 'is_weekend', 'is_night', 'click_intensity'],
      dtype='object')>


In [35]:
df_model = df[['ip', 'app', 'device', 'os', 'channel',
       'is_attributed', 'day', 'hour', 'minute', 'second',
       'time_to_conversion', 'device_os_combination', 'device_os_count',
       'ip_click_count', 'channel_click_count',
       'time_since_last_click', 'is_weekend', 'is_night']]

In [45]:
df_model.fillna(0, inplace = True)
nans = df_model.isna().sum()
print(nans)
df_model

ip                       0
app                      0
device                   0
os                       0
channel                  0
is_attributed            0
day                      0
hour                     0
minute                   0
second                   0
time_to_conversion       0
device_os_combination    0
device_os_count          0
ip_click_count           0
channel_click_count      0
time_since_last_click    0
is_weekend               0
is_night                 0
dtype: int64


,ip,app,device,os,channel,is_attributed,day,hour,minute,second,time_to_conversion,device_os_combination,device_os_count,ip_click_count,channel_click_count,time_since_last_click,is_weekend,is_night
1705658,0,10,1,46,20,1,9,1,43,8,34.0,1_46,19572,9,60114,0.0,0,1
1706762,0,10,1,42,20,1,9,1,44,53,34.0,1_42,8749,9,60114,105.0,0,1
2169455,0,10,1,4,20,1,9,12,45,57,32.0,1_4,11043,9,60114,39664.0,0,0
2170017,0,10,1,30,20,1,9,12,46,46,4.0,1_30,16714,9,60114,49.0,0,0
2182338,0,5,1,30,20,1,9,13,3,42,43.0,1_30,16714,9,60114,1016.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938032,260234,19,5,29,59,1,9,6,58,55,42.0,5_29,3029,1,139698,0.0,0,0
1972339,260235,34,1,19,73,1,9,7,48,5,184.0,1_19,509743,2,8848,0.0,0,0
2209994,260235,3,1,22,13,0,9,13,41,51,0.0,1_22,85200,2,2052,21226.0,0,0
2197432,260236,19,0,21,59,1,9,13,24,39,50.0,0_21,15215,1,139698,0.0,0,0


In [46]:
df_model.dtypes

ip                         int64
app                        int64
device                     int64
os                         int64
channel                    int64
is_attributed              int64
day                        uint8
hour                       uint8
minute                     uint8
second                     uint8
time_to_conversion       float64
device_os_combination     object
device_os_count            int64
ip_click_count             int64
channel_click_count        int64
time_since_last_click    float64
is_weekend                 int32
is_night                   int32
dtype: object

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# Split data
X = df_model.drop(columns=['is_attributed'])
y = df_model['is_attributed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

logreg = LogisticRegression(class_weight='balanced', random_state=42)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
y_pred_prob = logreg.predict_proba(X_test)[:, 1]

# Evaluate
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553115
           1       1.00      1.00      1.00    137054

    accuracy                           1.00    690169
   macro avg       1.00      1.00      1.00    690169
weighted avg       1.00      1.00      1.00    690169

ROC-AUC Score: 0.9996875720880695


In [50]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# Split data
X = df_model.drop(columns=['is_attributed'])
y = df_model['is_attributed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Oversample using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Logistic Regression
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train_resampled, y_train_resampled)

# Predictions
y_pred = logreg.predict(X_test)
y_pred_prob = logreg.predict_proba(X_test)[:, 1]

# Evaluate
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553115
           1       1.00      1.00      1.00    137054

    accuracy                           1.00    690169
   macro avg       1.00      1.00      1.00    690169
weighted avg       1.00      1.00      1.00    690169

ROC-AUC Score: 0.9996508688373219
